In [142]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [0]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

%matplotlib inline
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np
from glob import glob
import seaborn as sns

In [0]:
import zipfile as zipfile
zip_ref=zipfile.ZipFile("/gdrive/My Drive/train.zip",'r')

In [0]:
zip_ref.extractall("/tmp")

In [0]:
zip_ref=zipfile.ZipFile("/gdrive/My Drive/test.zip",'r')

In [0]:
zip_ref.extractall("/tmp")

In [174]:
!ls "/tmp/train"

 Black-grass	    'Common wheat'	'Scentless Mayweed'	      test
 Charlock	    'Fat Hen'		'Shepherds Purse'
 Cleavers	    'Loose Silky-bent'	'Small-flowered Cranesbill'
'Common Chickweed'   Maize		'Sugar beet'


In [0]:
def get_images(directory):
    Images = []
    Labels = []
    for dir_name in os.listdir(directory): 
        for image_file in os.listdir(directory+dir_name):
            image = cv2.imread(directory+dir_name+r'/'+image_file)
            if image is not None:
                image = cv2.resize(image,(51,51),)
                Images.append(image)
                Labels.append(dir_name)
    return Images, Labels

In [0]:
Images, Labels = get_images('../tmp/train/')

In [0]:
labels = []
mapping = { 'Sugar beet': 0, 'Fat Hen': 1, 'Scentless Mayweed' : 2, 'Charlock' : 3,
           'Small-flowered Cranesbill': 4, 'Maize': 5, 'Shepherds Purse' :6, 'Common wheat': 7,
           'Common Chickweed': 8, 'Cleavers': 9, 'Loose Silky-bent' : 10, 'Black-grass': 11 ,'test':12}
for label in Labels:  
    labels.append(mapping[label])
del Labels

In [178]:
Images[0].shape

(51, 51, 3)

In [0]:
Images = np.reshape(Images,(-1,51,51,3))
Labels = np.array(labels)

In [180]:
print("Shape of training data: ", Images.shape)
print("Shape of labels data: ", Labels.shape)

Shape of training data:  (5544, 51, 51, 3)
Shape of labels data:  (5544,)


In [0]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(Images, Labels, test_size=.2, random_state=42, stratify = Labels)

In [0]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train,num_classes=13)
y_val = np_utils.to_categorical(y_val,num_classes=13)

In [0]:
import tensorflow.keras.optimizers as Optimizer
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAvgPool2D, GlobalMaxPooling2D, Concatenate
from tensorflow.keras.models import Model
from keras.models import Sequential
import keras
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.convolutional import Conv2D,MaxPooling2D

In [0]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_train/=255
x_val/=255

In [190]:
input_shape = (51,51,3)
model = Sequential() 
model.add(Conv2D(256, kernel_size=(6, 6), strides=(1, 1), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2))) 
model.add(Flatten())
model.add(Dense(400, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(75, activation='relu'))
model.add(Dense(13, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 46, 46, 256)       27904     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 23, 23, 256)       0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 21, 21, 128)       295040    
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 10, 10, 128)       0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 12800)             0         
_________________________________________________________________
dense_36 (Dense)             (None, 400)               5120400   
_________________________________________________________________
dropout_15 (Dropout)         (None, 400)               0         
__________

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

In [193]:
batch_size=100
epochs = 150

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_val, y_val),callbacks=[])


Train on 4435 samples, validate on 1109 samples
Epoch 1/150
4435/4435 [==============================] - 2s 430us/step - loss: 0.9509 - acc: 0.6731 - val_loss: 1.1085 - val_acc: 0.6141
Epoch 2/150
4435/4435 [==============================] - 2s 415us/step - loss: 0.9312 - acc: 0.6760 - val_loss: 1.1085 - val_acc: 0.6123
Epoch 3/150
4435/4435 [==============================] - 2s 415us/step - loss: 0.9216 - acc: 0.6731 - val_loss: 1.1381 - val_acc: 0.6069
Epoch 4/150
4435/4435 [==============================] - 2s 417us/step - loss: 0.9177 - acc: 0.6679 - val_loss: 1.1011 - val_acc: 0.6267
Epoch 5/150
4435/4435 [==============================] - 2s 415us/step - loss: 0.9079 - acc: 0.6715 - val_loss: 1.1762 - val_acc: 0.6023
Epoch 6/150
4435/4435 [==============================] - 2s 417us/step - loss: 0.9129 - acc: 0.6762 - val_loss: 1.0826 - val_acc: 0.6276
Epoch 7/150
4435/4435 [==============================] - 2s 417us/step - loss: 0.8892 - acc: 0.6893 - val_loss: 1.6529 - val_acc: 

In [0]:
def get_test_images(directory):
    Images = []
    Image_names = []
    for image_file in os.listdir(directory):
        Image_names.append(image_file)
        image = cv2.imread(directory+r'/'+image_file)
        if image is not None:
            image = cv2.resize(image,(51,51),)
            Images.append(image)
    return Images, Image_names

In [195]:
test_images, image_names = get_test_images('../tmp/test/')
test_images = np.array(test_images)
print(test_images.shape)

(794, 51, 51, 3)


In [204]:
predictions = model.predict(test_images)
predictions = np.argmax(predictions, axis = 1)
predictions.shape

(794,)

In [0]:
labelled_predictions = []
mapping = {0: 'Sugar beet',1:'Fat Hen' ,2: 'Scentless Mayweed',3:  'Charlock', 
        4:'Small-flowered Cranesbill', 5:'Maize' ,
        6: 'Shepherds Purse' ,7:'Common wheat' ,8:'Common Chickweed' ,
        9:'Cleavers' ,10:'Loose Silky-bent'  ,11: 'Black-grass'}
for pred in predictions:
    if pred==12 or pred==0 or pred==10:
        continue  
    labelled_predictions.append(mapping[pred])


In [0]:
labelled_predictions = np.array(labelled_predictions)

In [208]:
# Evaluate the model on test set
score = model.evaluate(x_train, y_train, verbose=0)
# Print test accuracy
print('\n', 'Train accuracy:', score[1])


 Train accuracy: 0.9652762118831966


In [230]:
# Evaluate the model on test set
score = model.evaluate(x_val,y_val, verbose=0)
# Print test accuracy
print('\n', 'Validation accuracy:', score[1])


 Validation accuracy: 0.6663660959578291
